# Original model prediction

In [7]:
import tcrgp
import pickle
import ast
import csv
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
plt.style.use('ggplot')

In [18]:
def fix(datafile,resultpath):
    train=pd.read_csv(datafile)
    train['TRBV']=''
    train['TRBJ']=''
    train['cdr3a']=''
    epitope = pd.unique(train['Epitope'])
    for i in epitope:
        df_train = train[train['Epitope'] == i].sample(frac=1, random_state=42)
        df_train['cdr3a']='NaN'
        df_train=df_train[['Epitope','Affinity','TRBV','TRBJ','cdr3a','CDR3B']]
        df_train = df_train.rename(columns={'Epitope': 'epitope', 'CDR3B': 'cdr3b'})
        df_train.loc[df_train['Affinity'] == 0, 'epitope'] = 'none'
        df_train.to_csv(resultpath+"10000_"+i+".csv")
        
import pandas as pd
data=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/time/data_5000.csv")
target_rows = 10000
original_rows = len(data)
repeat_times = target_rows // original_rows + 1
df_repeated = pd.concat([data] * repeat_times, ignore_index=True)
df_repeated = df_repeated.head(target_rows)
df_repeated.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/time/10000_prediciton.csv")

datafile= "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/time/10000_prediciton.csv"   
resultpath="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/time/"
fix(datafile,resultpath)        

In [ ]:
def fix(datafile,resultpath):
    train=pd.read_csv(datafile)
    train['TRBV']=''
    train['TRBJ']=''
    train['cdr3a']=''
    epitope = pd.unique(train['Epitope'])
    for i in epitope:
        df_train = train[train['Epitope'] == i].sample(frac=1, random_state=42)
        df_train['cdr3a']='NaN'
        df_train=df_train[['Epitope','Affinity','TRBV','TRBJ','cdr3a','CDR3B']]
        df_train = df_train.rename(columns={'Epitope': 'epitope', 'CDR3B': 'cdr3b'})
        df_train.loc[df_train['Affinity'] == 0, 'epitope'] = 'none'
        df_train.to_csv(resultpath+i+".csv")
        
import pandas as pd
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
        datafile="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/"+i+'_'+j+'.csv'
        resultpath="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/10cross_validation/Prediction/all/seen/three/"+i+'_'+j
        fix(datafile,resultpath)

In [3]:
def validation_main(index_path,testfile_path,modelfile_path,result_path):
    index = pd.read_csv(index_path)
    epitope=pd.unique(index['Epitope'])
    all_results = pd.DataFrame()   
    for i in epitope:
        with open(modelfile_path+'model_vdj_'+i+'_cdr3b','rb') as f:
            params = pickle.load(f)
        preds = tcrgp.predict(testfile_path+i+'.csv',params)
        data = pd.read_csv(testfile_path + i + '.csv')
        probability = data[['epitope', 'cdr3b', 'Affinity']]
        probability = probability.rename(columns={'Affinity': 'y_true'})
        probability['y_prob'] = preds
        probability['y_pred'] = probability['y_prob'].apply(lambda x: 1 if x >= 0.5 else 0)
        probability['Epitope'] = probability['epitope'].str.replace('none', i)
        all_results = all_results.append(probability)
    all_results.to_csv(result_path+'probability.csv', index=False)

In [ ]:
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
        index_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/"+i+'_'+j+'.csv'
        modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/model/Original/"
        testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/10cross_validation/Prediction/all/seen/three/"+i+'_'+j
        result_path="./result/Original/seen/three/"+i+'_'+j
        validation_main(index_path,testfile_path,modelfile_path,result_path)
        

In [6]:
import pandas as pd
data=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/Prediction/all/seen/three/neg_pos/1_health.csv")
target_rows = 1000
original_rows = len(data)
repeat_times = target_rows // original_rows + 1
df_repeated = pd.concat([data] * repeat_times, ignore_index=True)
df_repeated = df_repeated.head(target_rows)
df_repeated.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/time/1000000_prediciton.csv")


# Model retraining

In [ ]:
def data_preprocessing(datafile,name,resultpath):                        
    train=pd.read_csv(datafile)
    epitope = pd.unique(train['Epitope'])
    for i in epitope:
        df_train = train[train['Epitope'] == i].sample(frac=1, random_state=42)
        df_train['cdr3a']='NaN'
        df_train=df_train[['Epitope','Affinity','TRBV','TRBJ','cdr3a','CDR3B']]
        df_train = df_train.rename(columns={'Epitope': 'epitope', 'CDR3B': 'cdr3b'})
        df_train.loc[df_train['Affinity'] == 0, 'epitope'] = 'none'
        df_train.to_csv(resultpath+i+"_train.csv")

        
database=['healthy','patient']
name=['1_1','1_2','1_4','1_6','1_8']
name1=['1','2','3','4','5']
for i in database:
    for j in name:
        for k in name1:
            datafile="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+j+"train.csv"
            resultpath="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/10cross_validation/pair50/"+i+"/"+k+'_'+j
            data_preprocessing(datafile,j,resultpath)

        
database=['Antigen_specificity']
name=['1_1','1_2','1_4']
name1=['1','2','3','4','5']
for i in database:
    for j in name:
        for k in name1:
            datafile="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+j+"train.csv"
            resultpath="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/10cross_validation/pair50/"+i+"/"+k+'_'+j
            data_preprocessing(datafile,j,resultpath)

database=['healthy','patient','Antigen_specificity']
name=['1_1']
name1=['1','2','3','4','5']
pair=['more300','300','200','100','10']
for i in database:
    for j in name:
        for k in name1:
            for l in pair:
                datafile="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j+"train.csv"
                resultpath="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j
                data_preprocessing(datafile,j,resultpath)

def data_preprocessing(datafile,name,resultpath):                        
    train=pd.read_csv(datafile)
    epitope = pd.unique(train['Epitope'])
    for i in epitope:
        df_train = train[train['Epitope'] == i].sample(frac=1, random_state=42)
        df_train['cdr3a']='NaN'
        df_train=df_train[['Epitope','Affinity','TRBV','TRBJ','cdr3a','CDR3B']]
        df_train = df_train.rename(columns={'Epitope': 'epitope', 'CDR3B': 'cdr3b'})
        df_train.loc[df_train['Affinity'] == 0, 'epitope'] = 'none'
        df_train.to_csv(resultpath+i+"_test.csv")

        
database=['healthy','patient','Antigen_specificity']
name=['1_1']
name1=['1','2','3','4','5']
for i in database:
    for j in name:
        for k in name1:
            datafile="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+j+"test.csv"
            resultpath="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/10cross_validation/pair50/"+i+"/"+k+'_'+j
            data_preprocessing(datafile,j,resultpath)

        
            
database=['healthy','patient','Antigen_specificity']
name=['1_1']
name1=['1','2','3','4','5']
for i in database:
    for j in name:
        for k in name1:
            datafile="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/more300/"+i+"/"+k+'_'+j+"test.csv"
            resultpath="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/10cross_validation/pair300/more300/"+i+"/"+k+'_'+j
            data_preprocessing(datafile,j,resultpath)
                        
                
def data_preprocessing(datafile,name,resultpath):                        
    train=pd.read_csv(datafile)
    epitope = pd.unique(train['Epitope'])
    for i in epitope:
        df_train = train[train['Epitope'] == i].sample(frac=1, random_state=42)
        df_train['cdr3a']='NaN'
        df_train=df_train[['Epitope','Affinity','TRBV','TRBJ','cdr3a','CDR3B']]
        df_train = df_train.rename(columns={'Epitope': 'epitope', 'CDR3B': 'cdr3b'})
        df_train.loc[df_train['Affinity'] == 0, 'epitope'] = 'none'
        df_train.to_csv(resultpath+i+"_Validation.csv")

database=['healthy','patient','Antigen_specificity']
name=['1_1'']
name1=['1','2','3','4','5]
for i in database:
    for j in name:
        for k in name1:
            datafile="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+j+"Validation.csv"
            resultpath="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/10cross_validation/Validation/"+i+"/"+k+'_'+j
            data_preprocessing(datafile,j,resultpath)

            

In [4]:
import pandas as pd
import pickle
import tcrgp

def train_main(index_path,trainfile_path, testfile_path,save_model_path,esult_path):
    subsmat = tcrgp.subsmatFromAA2('HENS920102')
    pc_blo = tcrgp.get_pcs(subsmat, d=21) 
    index = pd.read_csv(index_path)
    epitope = pd.unique(index['Epitope'])
    all_results wq= pd.DataFrame()   
    for i in epitope:
        auc, params = tcrgp.train_classifier(trainfile_path + i + '_train.csv', 'human', i, pc_blo,
                                                  cdr_types=[[], ['cdr3']], m_iters=20, lr=0.005, nZ=0, mbs=0, lmax3=18,
                                                  va=None, vb=None, cdr3a=None, cdr3b='cdr3b', epis='epitope')
        with open(save_model_path + i, 'wb') as f:
            pickle.dump(params, f)
        preds = tcrgp.predict(testfile_path + i + '_test.csv', params)
        data = pd.read_csv(testfile_path + i + '_test.csv')
        probability = data[['epitope', 'cdr3b', 'Affinity']]
        probability = probability.rename(columns={'Affinity': 'y_true'})
        probability['y_prob'] = preds
        probability['y_pred'] = probability['y_prob'].apply(lambda x: 1 if x >= 0.5 else 0)
        probability['epitope'] = probability['epitope'].str.replace('none', i)
        all_results = all_results.append(probability)
    all_results.to_csv(result_path+'probability.csv', index=False)

# pair50

In [ ]:
database=['healthy','patient','Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name:
        for k in name1:
            index_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+j+"train.csv"
            trainfile_path ="data/10cross_validation/pair50/"+i+"/"+k+'_'+j
            testfile_path="data/10cross_validation/pair50/"+i+"/"+k+'_'+j
            save_model_path="./model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="./result/pair50/seen/test/"+i+"/"+k+'_'+j
            train_main(index_path,trainfile_path,testfile_path,save_model_path,result_path)      

# pair300

In [ ]:
pair=['more300','300','200','100','10']            
database=['healthy','patient','Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name:
        for k in name1:
            for l in pair: 
                index_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j+"train.csv"
                trainfile_path ="data/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j
                testfile_path="data/10cross_validation/pair300/more300/"+i+"/"+k+'_'+j
                save_model_path="./model/Retrain/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                result_path="./result/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                train_main(index_path,trainfile_path,testfile_path,save_model_path,result_path)



/home/luyanping/miniconda3/envs/TCRGP3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (5,6,7,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/luyanping/miniconda3/envs/TCRGP3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (5,6,7,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/luyanping/miniconda3/envs/TCRGP3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (5,6,7,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/luyanping/miniconda3/envs/TCRGP3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (5,6,7,8,10) have mixed types.Specify dtype option on imp

In [3]:
pair=['more300','300','200','100','10']            
database=['healthy','patient','Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name:
        for k in name1:
            for l in pair: 
                index_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j+"train.csv"
                trainfile_path ="data/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j
                testfile_path="data/10cross_validation/pair300/more300/"+i+"/"+k+'_'+j
                save_model_path="./model/Retrain/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                result_path="./result/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                train_main(index_path,trainfile_path,testfile_path,save_model_path,result_path)



/home/luyanping/miniconda3/envs/TCRGP3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (5,6,7,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/luyanping/miniconda3/envs/TCRGP3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (5,6,7,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
def validation_main(index_path,testfile_path,modelfile_path,result_path):
    index = pd.read_csv(index_path)
    epitope=pd.unique(index['Epitope'])
    all_results = pd.DataFrame()   
    for i in epitope:
        with open(modelfile_path+i,'rb') as f:
            params = pickle.load(f)
        preds = tcrgp.predict(testfile_path+i+'_Validation.csv',params)
        data = pd.read_csv(testfile_path+i+'_Validation.csv')
        probability = data[['epitope', 'cdr3b', 'Affinity']]
        probability = probability.rename(columns={'Affinity': 'y_true'})
        probability['y_prob'] = preds
        probability['y_pred'] = probability['y_prob'].apply(lambda x: 1 if x >= 0.5 else 0)
        probability['Epitope'] = probability['epitope'].str.replace('none', i)
        all_results = all_results.append(probability)
    all_results.to_csv(result_path+'probability.csv', index=False)

# validaiton

In [ ]:
import pandas as pd
import numpy as np
database=['healthy','patient','Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name:
        for k in name1:
            index_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+"1_1Validation.csv"
            testfile_path="./data/10cross_validation/Validation/"+i+"/"+k+'_'+j
            modelfile_path="./model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="./result/pair50/seen/validation/"+i+"/"+k+'_'+j
            validation_main(index_path,testfile_path,modelfile_path,result_path)     

In [ ]:
import pandas as pd
import numpy as np
database=['patient','Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name:
        for k in name1:
            index_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+"1_1Validation.csv"
            testfile_path="./data/10cross_validation/Validation/"+i+"/"+k+'_'+j
            modelfile_path="./model/Retrain/pair50/"+i+"/"+k+'_'+j
            result_path="./result/pair50/seen/validation/"+i+"/"+k+'_'+j
            validation_main(index_path,testfile_path,modelfile_path,result_path)     

# Top5 parameter adjustment

In [ ]:
import pandas as pd
reap=['reap1','reap2','reap3','reap4','reap5']
name=['top1','top2','top3','top4','top5']
number= ['3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50']
for i in reap:
    for j in name:
        for k in number:
            datafile="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+k+'.csv'   
            resultpath="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/10cross_validation/top5/healthy/"+i+"_" +j+"_"+k
            fix(datafile,resultpath)

def fix(datafile,resultpath):
    train=pd.read_csv(datafile)
    train['TRBV']=''
    train['TRBJ']=''
    train['cdr3a']=''
    epitope = pd.unique(train['Epitope'])
    for a in epitope:
        df_train = train[train['Epitope'] == a].sample(frac=1, random_state=42)
        df_train['cdr3a']='NaN'
        df_train=df_train[['Epitope','Affinity','TRBV','TRBJ','cdr3a','CDR3B']]
        df_train = df_train.rename(columns={'Epitope': 'epitope', 'CDR3B': 'cdr3b'})
        df_train.loc[df_train['Affinity'] == 0, 'epitope'] = 'none'
        df_train.to_csv(resultpath+a+"test.csv")
import pandas as pd
reap=['reap1','reap2','reap3','reap4','reap5']
name=['top1','top2','top3','top4','top5']
number= ['3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50']
for i in reap:
    for j in name:
        for k in number:
            datafile="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+'test.csv'
            resultpath="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/data/10cross_validation/top5/healthy/"+i+"_" +j
            fix(datafile,resultpath)
 

In [ ]:
reap=['reap1','reap2','reap3','reap4','reap5']
name=['top1','top2','top3','top4','top5']
number= ['3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50']
for i in reap:
    for j in name:
        for k in number:
            index_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+k+'.csv'
            trainfile_path ="data/10cross_validation/top5/healthy/"+i+'_'+j+'_'+k
            testfile_path="data/10cross_validation/top5/healthy/"+i+'_'+j+'_'+k
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/result/top5/seen/healthy/"+i+'_'+j+'_'+k
            train_main(index_path,trainfile_path, testfile_path,result_path)  